In [6]:
class Bracket:
    def __init__(self,bestOf,size,dice):
        self.bestOf = bestOf
        self.size = size
        self.dice = dice
        number = len(self.dice)
        self.players = []
        
        for i in range(number):
            self.players.append(Player(self.dice))
            
        for i in range(number):
            p = self.players[i]
            d = self.dice[i]
            p.chooseMain(d)
            
    def show(self):
        for p in self.players:
            p.showWins()
            
    def tournament(self): 
        bracket = self.players.copy()
        winBracket = []
        loseBracket = []
        dead = []
        random.shuffle(bracket)
        
        while len(bracket) > self.size:
            bracket.pop()
        
        winBracket, loseBracket = Split(bracket, self.bestOf)
        
        while (len(winBracket) != 1) or (len(loseBracket) != 1): 
            random.shuffle(winBracket)
            random.shuffle(loseBracket)
            
            bracket = loseBracket.copy()
            
            loseBracket, dead = Split(bracket, self.bestOf)
            
            bracket = winBracket.copy()
            winBracket, bracket = Split(bracket, self.bestOf)
            for p in bracket:
                loseBracket.append(p)

            random.shuffle(loseBracket)
            
            bracket = loseBracket.copy()
            loseBracket, dead = Split(bracket, self.bestOf)
        
        playerA = winBracket[0]
        playerB = loseBracket[0]
        
        winner, loser = Match(playerA,playerB,self.bestOf)
        
        if winner == playerB:
                winner, loser = Match(winner,loser,self.bestOf)
        
        winner.winTournament()
        
    def series(self, iterations):
        for i in range(iterations):
            self.tournament()
            


In [7]:
def Split(bracket, bestOf):
    winners = []
    losers = []
    
    while len(bracket) > 0:
        playerA = bracket.pop()
        playerB = bracket.pop()
        winner,loser = Match(playerA,playerB,bestOf)
        winners.append(winner)
        losers.append(loser)
        
    return winners, losers

In [10]:
def Match(playerA,playerB,bestOf):
    
    targetScore = (bestOf+1)/2
    scoreA = 0
    scoreB = 0
    
    while (scoreA < targetScore) and (scoreB < targetScore):
        if Game(playerA,playerB) == playerA:
            scoreA += 1
        else:
            scoreB += 1
            
    if scoreA == targetScore:
        return playerA,playerB
    else:
        return playerB,playerA
    

In [11]:
def Game(playerA,playerB):
    
    rollA = playerA.roll()
    rollB = playerB.roll()
    
    while rollA == rollB:    
        rollA = playerA.roll()
        rollB = playerB.roll()
    
    if rollA>rollB:
        return playerA
    else:
        return playerB

In [12]:
def simulateSeries(dice, size):
    try:
        f = open("Tournament Series.txt",'w')
        
        f.write("size\tBest of")
        for d in dice:
            f.write("\t")
            f.write(str(d.sides))
        f.write("\n")
        
        for s in range(1,9):
            s = 2**s
            print(s)
            
            for b in [1,3,5,7]:
                bra = Bracket(b,s,everyDie.dice)
                bra.series(size)
                
                f.write(str(s) + "\t" + str(b))
                
                for p in bra.players:
                    f.write("\t")
                    f.write(str(p.wins))
                f.write("\n")       
        
    finally:
        print("done!")
        f.close()